# Getting basins for all NVE discharge-measuring stations
This notebook assumes that NVE info has been downloaded using the get_nve_data.ipynb notebook in this folder

## Loading required packages and getting credentials to connect to the database

In [1]:
import requests
import json
import matplotlib.pyplot as plt
import datetime
import pandas as pd
from datetime import datetime
from datetime import timedelta
from fabric import Connection
import psycopg2
import gmaps
import pandas.io.sql as sqlio
import getpass
from io import StringIO
import paramiko
import sys
from time import sleep

sys.path.insert(0, "/home/jovyan/watexr/PROGNOS/")
from prognos_tools.gce_light import gce_api as gce

key = getpass.getpass('mobiserver password: ')
cloudKey = getpass.getpass('vault password: ')
#Querying necessary tokens
def query(query,fetch=True):
    with psycopg2.connect(user='jose-luis', host='mobiserver.niva.no', port=5432, database='vault',password=key) as db:
        with db.cursor() as cursor :
            cursor.execute(query)
            if fetch:
                result = sqlio.read_sql_query(query, db)
                return result
            
gmapsKey = query('''select niva.getToken('gmaps','{}');'''.format(cloudKey)).iloc[0,0]
sshKey = query('''select niva.getToken('geonorway_ssh_key','{}');'''.format(cloudKey)).iloc[0,0]
cloudKey = json.loads(query('''select niva.getToken('gce_access','{}');'''.format(cloudKey)).iloc[0,0])
gmaps.configure(gmapsKey)
not_really_a_file = StringIO(sshKey)
private_key = paramiko.RSAKey.from_private_key(not_really_a_file)
del key,sshKey

mobiserver password:  ·······
vault password:  ···············


## Starting the instance containing the database using the google compute engine api

In [2]:
#Check status of instance
properties = {'project'      : 'nivacatchment',
             'zone'         : 'europe-north1-a',
             'instanceType' : "n1-standard-4",
             'instanceName' : "dtm10",
             'username'     : "jose-luis",
             }

cloud = gce(properties, cloudKey)
del cloudKey

#Getting instance info
cloud.CommonCalls['custom'] = '''https://compute.googleapis.com/compute/v1/projects/{project}/zones/{zone}/instances/{instanceName}'''
info = cloud.get('custom')
display(info['status'])
#If instance is stopped, start it
if info['status'] != 'RUNNING':
    cloud.CommonCalls['custom'] = '''https://compute.googleapis.com/compute/v1/projects/{project}/zones/{zone}/instances/{instanceName}/start'''
    info = cloud.post('custom')
    display(info['status'])
    cloud.CommonCalls['custom'] = '''https://compute.googleapis.com/compute/v1/projects/{project}/zones/{zone}/instances/{instanceName}'''
    info = cloud.get('custom')
    while info['status'] != 'RUNNING':
           sleep(2)
           info = cloud.get('custom')
          
geonorway = info['networkInterfaces'][0]['accessConfigs'][0]['natIP']    
    
display("IP of the instance containing the database {}:".format(geonorway))

#The config below can be used to connect to the instance using fabric's Connection
config =  {'host' : geonorway, 'user': 'jose-luis', 'connect_kwargs': {'pkey': private_key } }

'RUNNING'

'IP of the instance containing the database 35.228.213.48:'

## Getting the coordinates of NVE's discharge-measuring stations and the basin they belong to

In [4]:
#Helper function to query the database and return a pandas dataframe if required.
def query(query,fetch=True):
    with psycopg2.connect(user='jose-luis', host=geonorway, port=5432, database='geonorway') as db:
        cursor = db.cursor()
        cursor.execute(query)
        if fetch:
            result = sqlio.read_sql_query(query, db)
            return result
        
#Schema where the nve data resides
schema = 'nveq'

#Querying the coordinates of the stations and the basin they are in:
sql = '''with bla as
(select stationid,stationname,st_transform(geom,3045) as geom from {}.stations)
select b.stationid,a.gid,a.vassomr,st_area(a.geom)/1e6 as area
from nedborfelt.vassdragsomr as a, bla as b 
where st_intersects(a.geom,b.geom);'''.format(schema)

a = query(sql)
display(a)

,stationid,gid,vassomr,area
0,1.198.0,1,Haldenvassdraget/Iddefjorden,2507.380211
1,1.200.0,1,Haldenvassdraget/Iddefjorden,2507.380211
2,1.48.0,1,Haldenvassdraget/Iddefjorden,2507.380211
3,1.49.0,1,Haldenvassdraget/Iddefjorden,2507.380211
4,1.50.0,1,Haldenvassdraget/Iddefjorden,2507.380211
...,...,...,...,...
597,97.5.0,97,"Storfjorden sør, Hjørundfjorden og Sykkylvsfjo...",868.995031
598,98.4.0,98,"Storfjorden sør, Sunnylvsfjorden og Geirangerf...",1003.641202
599,99.1.0,99,Tafjordvassdraget/Tafjorden og Norddalsfjorden...,643.801323
600,99.17.0,99,Tafjordvassdraget/Tafjorden og Norddalsfjorden...,643.801323


In [12]:
sql = '''with bla as
(select stationid,stationname,st_transform(geom,3045) as geom from {}.stations)
select b.stationid,b.stationname from bla as b 
where b.stationid not in (select b.stationid from bla as b, nedborfelt.vassdragsomr as a where st_intersects(a.geom,b.geom));'''.format(schema)

display(query(sql))

,stationid,stationname
0,19.244.0,Nye Jørundland


In [13]:
display(query('''select * from nveq.stations where stationid='19.244.0'  '''))

,sid,stationname,stationid,latitude,longitude,geom
0,241,Nye Jørundland,19.244.0,1.11498,5.61744,0101000020E61000000EF8FC3042781640F41ABB44F5D6...


In [28]:
display(query('''select metadata -> 'data'  from nveq.metadata   '''))

,?column?
0,None
1,None
2,None
3,None
4,None
...,...
598,None
599,None
600,None
601,None
